# Regression

In [1]:
import geopandas as gpd

blocks_gdf = gpd.read_parquet('./../data/blocks.parquet')

In [2]:
from blocksnet.relations import generate_adjacency_graph

adjacency_graph = generate_adjacency_graph(blocks_gdf, 10)

2025-04-02 15:54:02.412 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_nodes:10 - Generating nodes
2025-04-02 15:54:02.422 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_edges:15 - Generating edges
2025-04-02 15:54:04.058 | SUCCESS  | blocksnet.relations.adjacency.core:generate_adjacency_graph:34 - Adjacency graph successfully generated: 16320 nodes, 16036 edges


## Density

In [3]:
from blocksnet.machine_learning.regression import DensityRegressor

dr = DensityRegressor()

### Train

In [4]:
data = dr.get_train_data(blocks_gdf, adjacency_graph)

2025-04-02 15:54:07.357 | WARNING  | blocksnet.machine_learning.regression.density.schemas:_before_validate:35 - Not valid format. Trying to one hot from land_use column
2025-04-02 15:54:10.532 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:34 - Calculating usual features
2025-04-02 15:54:10.805 | INFO     | blocksnet.machine_learning.regression.density.core:_features_from_geometries:37 - Fitting the scaler


In [12]:
losses = dr.train(data, epochs=10_000)

Current loss : 0.064:  39%|███▉      | 3943/10000 [00:45<01:10, 86.37it/s]


KeyboardInterrupt: 

### Evaluate

In [13]:
result_df = dr.evaluate(blocks_gdf, adjacency_graph)

2025-04-02 15:56:18.474 | WARNING  | blocksnet.machine_learning.regression.density.schemas:_before_validate:35 - Not valid format. Trying to one hot from land_use column
2025-04-02 15:56:21.236 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:34 - Calculating usual features


In [14]:
blocks_gdf[['fsi', 'gsi', 'mxi']]

,fsi,gsi,mxi
id,,,
0,0.054497,0.017345,0.700000
1,0.444238,0.217986,0.211541
2,0.003234,0.003205,0.700000
3,0.235746,0.056437,0.660753
4,0.086291,0.025381,0.692085
...,...,...,...
16315,0.000000,0.000000,NaN
16316,0.000000,0.000000,NaN
16317,0.000000,0.000000,NaN


In [15]:
result_df

,fsi,gsi,mxi
id,,,
0,0.096210,0.096210,0.236500
1,0.039109,0.039109,0.046922
2,0.126786,0.126786,0.212395
3,0.421853,0.183958,0.434653
4,0.069512,0.069512,0.271729
...,...,...,...
16315,0.004094,0.004094,0.050733
16316,0.002831,0.002831,0.018675
16317,0.010259,0.010259,0.047761
